In [20]:
# Load data ke dalam data frame 
import pandas as pd

# Spesifikasi encoding diperlukan karena data tidak menggunakan UTF-8
df = pd.read_csv('scraping_azarine.csv', encoding='latin-1')

df.head()

,Review,Product,Rating,Username,Date
0,sipp,Azarine Cicamide Barrier Sunscreen Moisturiser...,bintang 5,V***i,1/30/2024
1,proses orderan san pengirimannya sesuai estima...,[Twin Pack] Azarine Ceraspray Sunscreen Spray ...,bintang 5,Frisca,1/30/2024
2,terima kasih sudah diterima dengan baik,Azarine Retinol Smooth Glowing Serum 20ml,bintang 5,ridwan,1/30/2024
3,tipe kulitku kombinasi tipe kulitku normal tip...,Azarine Acne Spot Serum 20ml,bintang 5,B***H,1/30/2024
4,Ini kurang oke menurut sy. Dipake ga bikin bib...,[Twin pack] Azarine Magic Colour Lip Serum 3.5...,bintang 4,Sabina,1/30/2024


In [21]:
# Drop kolom yang tidak terpakai
df = df.drop(df.iloc[:,1:], axis=1)

# Cek data teratas
df.head()

,Review
0,sipp
1,proses orderan san pengirimannya sesuai estima...
2,terima kasih sudah diterima dengan baik
3,tipe kulitku kombinasi tipe kulitku normal tip...
4,Ini kurang oke menurut sy. Dipake ga bikin bib...


PREPROCESSING

In [22]:
# PROSES CASE FOLDING (menjadikan semua huruf di kolom Review menjadi huruf kecil)
df['Review'] = df['Review'].str.lower()

print('Case Folding Result: \n')
df.head()

Case Folding Result: 



,Review
0,sipp
1,proses orderan san pengirimannya sesuai estima...
2,terima kasih sudah diterima dengan baik
3,tipe kulitku kombinasi tipe kulitku normal tip...
4,ini kurang oke menurut sy. dipake ga bikin bib...


In [23]:
# import hasil case folding menjadi csv
df.to_csv("casefolding_azarine.csv")

In [24]:
# CLEANING
import re # regex library

def remove_tweet_special(text):
    # remove tab, new line, and back slice
    text = text.replace('\\t'," ").replace('\\n', " ").replace('\\u', " ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df['Review'] = df['Review'].apply(remove_tweet_special)

# remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['Review'] = df['Review'].apply(remove_number)

# remove punctuation and replace with space
def remove_punctuation(text):
    return re.sub(r'[.,]', ' ', text)

df['Review'] = df['Review'].apply(remove_punctuation)

# remove punctuation
def remove_symbol(text):
    # Menghapus simbol-simbol tidak standar dan menggantinya dengan spasi
    cleaned_text = re.sub(r'[^\w\s]', ' ', text)
    
    # Menghapus multiple whitespace
    cleaned_text = re.sub('\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

df['Review'] = df['Review'].apply(remove_symbol)

# Menentukan ambang batas panjang string acak
threshold_length = 20

# Fungsi untuk menghapus string acak berdasarkan panjangnya
def hapus_string_acak_dengan_panjang(text):
    # Memeriksa panjang string dan menghapus jika melebihi ambang batas
    return ' '.join(word for word in text.split() if len(word) <= threshold_length)

# Menggunakan fungsi untuk menghapus string acak pada kolom 'Text'
df['Review'] = df['Review'].apply(hapus_string_acak_dengan_panjang)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['Review'] = df['Review'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['Review'] = df['Review'].apply(remove_whitespace_multiple)

# remove single char
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['Review'] = df['Review'].apply(remove_single_char)

def remove_laughter(text):
    laughter_patterns = r'\b((ha)+h*|(he)+h*|(hi)+h*|(wk)+w*k*|(eh)+e*|(ah)+a*|(ih)+i*|(kw)+k*w*|(hem)+m*)\b'
    return re.sub(laughter_patterns, '', text, flags=re.IGNORECASE)

df['Review'] = df['Review'].apply(remove_laughter)

print('Cleaning Result : \n') 
print(df.head())

Cleaning Result : 

                                              Review
0                                               sipp
1  proses orderan san pengirimannya sesuai estima...
2            terima kasih sudah diterima dengan baik
3  tipe kulitku kombinasi tipe kulitku normal tip...
4  ini kurang oke menurut sy dipake ga bikin bibi...


In [25]:
# import hasil cleaning menjadi csv
df.to_csv("cleaning_azarine.csv")

In [26]:
# REMOVE DUPLICATE
df = df.drop_duplicates()
df = df.reset_index(drop=True)
# Menghapus baris yang kosong
df = df.dropna(subset=['Review'])
# Menghapus baris yang hanya berisi spasi atau whitespace
df = df[df['Review'].str.strip() != '']
df.head()

,Review
0,sipp
1,proses orderan san pengirimannya sesuai estima...
2,terima kasih sudah diterima dengan baik
3,tipe kulitku kombinasi tipe kulitku normal tip...
4,ini kurang oke menurut sy dipake ga bikin bibi...


In [27]:
# import hasil remove duplicate menjadi csv
df.to_csv("removedup_azarine.csv")

In [28]:
# NORMALIZATION
import pandas as pd
import re

slang_dictionary = pd.read_csv('../colloquial-indonesian-lexicon2.csv')
slang_dict = pd.Series(slang_dictionary['formal'].values,index=slang_dictionary['slang']).to_dict()

slang_dictionary.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1.0,wow,elongasi,0,0
1,aminn,amin,1.0,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1.0,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1.0,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0.0,Birthday yg keberpa kak?,abreviasi,0,0


In [29]:
# Normalisasi kata menggunakan kamus colloquial-indonesian-lexicon2.csv
def Slangwords(text, slang_dict):
    for word in text.split():
        if word in slang_dict.keys():
            # menambahkan \b untuk menandakan batas kata di sekitar kata slang
            text = re.sub(r'\b{}\b'.format(re.escape(word)), slang_dict[word], text)
    
    text = re.sub('@[\w]+', '', text)
    return text

df['Review_norm'] = df['Review'].apply(lambda x: Slangwords(x, slang_dict))
print(df.head())

                                              Review  \
0                                               sipp   
1  proses orderan san pengirimannya sesuai estima...   
2            terima kasih sudah diterima dengan baik   
3  tipe kulitku kombinasi tipe kulitku normal tip...   
4  ini kurang oke menurut sy dipake ga bikin bibi...   

                                         Review_norm  
0                                                sip  
1  proses orderan san pengirimannya sesuai estima...  
2            terima kasih sudah diterima dengan baik  
3  tipe kulitku kombinasi tipe kulitku normal tip...  
4  ini kurang oke menurut saya dipakai enggak bik...  


In [30]:
# import hasil replace slang words menjadi csv
df.to_csv("normalization_azarine.csv")

In [31]:
# PROSES TOKENIZING (word_tokenize() untuk memecah string kedalam tokens)

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Tokenizing
# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['review_tokens'] = df['Review_norm'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df.head())

Tokenizing Result : 

                                              Review  \
0                                               sipp   
1  proses orderan san pengirimannya sesuai estima...   
2            terima kasih sudah diterima dengan baik   
3  tipe kulitku kombinasi tipe kulitku normal tip...   
4  ini kurang oke menurut sy dipake ga bikin bibi...   

                                         Review_norm  \
0                                                sip   
1  proses orderan san pengirimannya sesuai estima...   
2            terima kasih sudah diterima dengan baik   
3  tipe kulitku kombinasi tipe kulitku normal tip...   
4  ini kurang oke menurut saya dipakai enggak bik...   

                                       review_tokens  
0                                              [sip]  
1  [proses, orderan, san, pengirimannya, sesuai, ...  
2     [terima, kasih, sudah, diterima, dengan, baik]  
3  [tipe, kulitku, kombinasi, tipe, kulitku, norm...  
4  [ini, kurang, oke, menurut

In [32]:
# import hasil tokenizing menjadi csv
df.to_csv("tokenizing_azarine.csv")

In [33]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
# PROSES FILTERING (Stopword Removal) menggunakan library sastrawi
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# remove stopword pada list token
def stopwords_removal(words):
    filtered_words = [stopword.remove(w) for w in words]
    filtered_words = [word for word in filtered_words if word != '']  # Memfilter kata-kata kosong
    return filtered_words

df['review_tokens_SR'] = df['review_tokens'].apply(stopwords_removal) 

print(df.head())

                                              Review  \
0                                               sipp   
1  proses orderan san pengirimannya sesuai estima...   
2            terima kasih sudah diterima dengan baik   
3  tipe kulitku kombinasi tipe kulitku normal tip...   
4  ini kurang oke menurut sy dipake ga bikin bibi...   

                                         Review_norm  \
0                                                sip   
1  proses orderan san pengirimannya sesuai estima...   
2            terima kasih sudah diterima dengan baik   
3  tipe kulitku kombinasi tipe kulitku normal tip...   
4  ini kurang oke menurut saya dipakai enggak bik...   

                                       review_tokens  \
0                                              [sip]   
1  [proses, orderan, san, pengirimannya, sesuai, ...   
2     [terima, kasih, sudah, diterima, dengan, baik]   
3  [tipe, kulitku, kombinasi, tipe, kulitku, norm...   
4  [ini, kurang, oke, menurut, saya, dipakai, 

In [34]:
# import hasil stopword removal menjadi csv
df.to_csv("stopword_azarine.csv")

In [35]:
# PROSES LEMMATIZATION 
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['review_tokens_SR']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['review_tokens_stemmed'] = df['review_tokens_SR'].swifter.apply(get_stemmed_term)
print(df['review_tokens_stemmed'])
print(df.head())

987
------------------------
sip : sip
proses : proses
orderan : order
san : san
pengirimannya : kirim
sesuai : sesuai
estimasi : estimasi
pengemasannya : emas
aman : aman
banget : banget
kulitku : kulit
cukup : cukup
sensitif : sensitif
sering : sering
pakai : pakai
makeup : makeup
jadi : jadi
mau : mau
coba : coba
sunscreen : sunscreen
spray : spray
menarik : tarik
cooling : cooling
sensationnya : sensationnya
semoga : moga
cocok : cocok
thank : thank
you : you
azarine : azarine
terima : terima
kasih : kasih
diterima : terima
baik : baik
tipe : tipe
kombinasi : kombinasi
normal : normal
kering : kering
berminyak : minyak
masalah : masalah
jerawat : jerawat
pori : pori
besar : besar
kurang : kurang
oke : oke
dipakai : pakai
enggak : enggak
bikin : bikin
bibir : bibir
lembab : lembab
malah : malah
kasar : kasar
warnanya : warna
merah : merah
menor : menor
approved : approved
sawo : sawo
matang : matang
hitam : hitam
kelihatan : lihat
dicoba : coba
takut : takut
ternyata : nyata
kulit :

Pandas Apply: 100%|██████████| 372/372 [00:00<?, ?it/s]

0                                                  [sip]
1      [proses, order, san, kirim, sesuai, estimasi, ...
2                          [terima, kasih, terima, baik]
3      [tipe, kulit, kombinasi, tipe, kulit, normal, ...
4      [kurang, oke, pakai, enggak, bikin, bibir, lem...
                             ...                        
368    [kecewa, sama, emas, kardus, peyok, dalem, eng...
369    [ringan, banget, tipe, tekstur, moist, suka, c...
370    [suka, ingredientsnya, mudah, cocok, terima, k...
371         [tipe, kulit, minyak, tipe, kulit, sensitif]
372                                              [bagus]
Name: review_tokens_stemmed, Length: 372, dtype: object
                                              Review  \
0                                               sipp   
1  proses orderan san pengirimannya sesuai estima...   
2            terima kasih sudah diterima dengan baik   
3  tipe kulitku kombinasi tipe kulitku normal tip...   
4  ini kurang oke menurut sy dipake g

In [44]:
# Menghapus baris yang kosong
df = df.dropna(subset=['review_tokens_stemmed'])
# Menghapus baris yang hanya berisi spasi atau whitespace
df = df[df['review_tokens_stemmed'].str.strip() != '']
# Menghapus baris yang memiliki list kosong
df = df[df['review_tokens_stemmed'].apply(lambda x: x != [])]
df.head(15)

,Review,Review_norm,review_tokens,review_tokens_SR,review_tokens_stemmed
0,sipp,sip,[sip],[sip],[sip]
1,proses orderan san pengirimannya sesuai estima...,proses orderan san pengirimannya sesuai estima...,"[proses, orderan, san, pengirimannya, sesuai, ...","[proses, orderan, san, pengirimannya, sesuai, ...","[proses, order, san, kirim, sesuai, estimasi, ..."
2,terima kasih sudah diterima dengan baik,terima kasih sudah diterima dengan baik,"[terima, kasih, sudah, diterima, dengan, baik]","[terima, kasih, diterima, baik]","[terima, kasih, terima, baik]"
3,tipe kulitku kombinasi tipe kulitku normal tip...,tipe kulitku kombinasi tipe kulitku normal tip...,"[tipe, kulitku, kombinasi, tipe, kulitku, norm...","[tipe, kulitku, kombinasi, tipe, kulitku, norm...","[tipe, kulit, kombinasi, tipe, kulit, normal, ..."
4,ini kurang oke menurut sy dipake ga bikin bibi...,ini kurang oke menurut saya dipakai enggak bik...,"[ini, kurang, oke, menurut, saya, dipakai, eng...","[kurang, oke, dipakai, enggak, bikin, bibir, l...","[kurang, oke, pakai, enggak, bikin, bibir, lem..."
5,tipe kulitku berminyak,tipe kulitku berminyak,"[tipe, kulitku, berminyak]","[tipe, kulitku, berminyak]","[tipe, kulit, minyak]"
6,tipe kulitku sensitif masalah kulitku jerawat,tipe kulitku sensitif masalah kulitku jerawat,"[tipe, kulitku, sensitif, masalah, kulitku, je...","[tipe, kulitku, sensitif, masalah, kulitku, je...","[tipe, kulit, sensitif, masalah, kulit, jerawat]"
7,gak approved untuk sawo matang bibir hitam krn...,enggak approved untuk sawo matang bibir hitam ...,"[enggak, approved, untuk, sawo, matang, bibir,...","[enggak, approved, sawo, matang, bibir, hitam,...","[enggak, approved, sawo, matang, bibir, hitam,..."
8,setelah dicoba takut gak cocok ternyata cocok ...,setelah dicoba takut enggak cocok ternyata coc...,"[setelah, dicoba, takut, enggak, cocok, ternya...","[dicoba, takut, enggak, cocok, ternyata, cocok...","[coba, takut, enggak, cocok, nyata, cocok, kul..."
9,selalu cocok sama produk azarine top dech,selalu cocok sama produk azarine top deh,"[selalu, cocok, sama, produk, azarine, top, deh]","[selalu, cocok, sama, produk, azarine, top, deh]","[selalu, cocok, sama, produk, azarine, top, deh]"


In [45]:
# import hasil preprocessing menjadi csv
df.to_csv("lemma_azarine.csv")